In [ ]:
import tensorflow as tf

import pandas as pd
import numpy as np
import random
import cv2

mnist = pd.read_csv('train.csv')

train_labels = mnist['label']

train_data = mnist.drop('label', 1)


#trainnn = cv2.imread("images/image1.png", 0).flatten()
training_data =np.zeros((160000, 784))
k=0

#importing augmented image data as training data
for i in range(32000):
    str1 = "images/image" +str(i) + ".png"
    str2 = "images/image" +str(i) + "__blur2.0.png"
    str3 = "images/image" +str(i) + "__noise0.03.png"
    str4 = "images/image" +str(i) + "__rot-30.png"
    str5 = "images/image" +str(i) + "__trans3_3.png"
    
    training_data[k] = cv2.imread(str1, 0).flatten()
    k=k+1
    training_data[k] = cv2.imread(str2, 0).flatten()
    k=k+1
    training_data[k] = cv2.imread(str3, 0).flatten()
    k=k+1
    training_data[k] = cv2.imread(str4, 0).flatten()
    k=k+1
    training_data[k] = cv2.imread(str5, 0).flatten()
    k=k+1
    
    

training_data = (training_data).astype('uint8')
training_data = training_data/256

training_data

In [ ]:
training_labels = np.zeros(160000)
k =0 
for i in range(32000):
    training_labels[k] = train_labels[i]
    k=k+1
    training_labels[k] = train_labels[i]
    k=k+1
    training_labels[k] = train_labels[i]
    k=k+1
    training_labels[k] = train_labels[i]
    k=k+1
    training_labels[k] = train_labels[i]
    k=k+1

training_labels[154000]
    

In [ ]:

np.shape(train_data)

labels_onehot = np.zeros((160000,10))
        
for i in range(160000): 
    for j in range(10):
        if j == training_labels[i]:
            labels_onehot[i,j] = 1
        else:
            labels_onehot[i,j] = 0



def next_batch(data, labels_onehot, batch_size):
        
        
        
        A = random.randint(0, 160000 - batch_size-1)
        B = A + batch_size
        xs = data[A:B]
        ys = labels_onehot[A:B]
        
    
    
        return xs, ys
    
        
labels_onehot
    

In [ ]:
n_classes = 10
batch_size = 256

x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')
keep_prob= tf.placeholder(tf.float32)



def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def maxpool2d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')



def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.truncated_normal([5,5,1,32], stddev = 0.1)),
               'W_conv2':tf.Variable(tf.truncated_normal([5,5,32,64], stddev = 0.1)),
               'W_fc1':tf.Variable(tf.truncated_normal([7*7*64,1024], stddev = 0.1)),
               'W_fc2':tf.Variable(tf.truncated_normal([1024,512], stddev = 0.1)),
               'out':tf.Variable(tf.truncated_normal([512, n_classes], stddev = 0.1))}

    biases = {'b_conv1':tf.Variable(tf.constant(0.1, shape = [32])),
               'b_conv2':tf.Variable(tf.constant(0.1, shape = [64])),
               'b_fc1':tf.Variable(tf.constant(0.1, shape = [1024])),
               'b_fc2':tf.Variable(tf.constant(0.1, shape = [512])),
               'out':tf.Variable(tf.constant(0.1, shape = [n_classes]))}

    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool2d(conv1)
    
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool2d(conv2)

    fc = tf.reshape(conv2,[-1, 7*7*64])
    
    fc1 = tf.nn.relu(tf.matmul(fc, weights['W_fc1'])+biases['b_fc1'])
    fc2 = tf.nn.relu(tf.matmul(fc1, weights['W_fc2'])+biases['b_fc2'])
    
    
    
    fc2 = tf.nn.dropout(fc2, keep_prob)
    
    output = tf.matmul(fc2, weights['out'])+biases['out']

    return output

def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(prediction, y))
    optimizer = tf.train.AdamOptimizer(1e-4).minimize(cost)
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    sess= tf.Session()
    
    sess.run(tf.global_variables_initializer())
    
    for i in range(35000):
        batch_xs, batch_ys = next_batch(training_data, labels_onehot, batch_size)
        
        sess.run(optimizer, feed_dict = {x: batch_xs, y: batch_ys, keep_prob: 0.2})
        if i%100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x:batch_xs, y: batch_ys, keep_prob: 0.5})
            print("step %d, training accuracy %g"%(i, train_accuracy))
    
    test = pd.read_csv('test.csv')
    a = test.as_matrix()
    a = (a/256)
    
    Imageid = np.linspace(1, 10000, num=10000, dtype = int)
    
    prediction1=tf.argmax(prediction,1)
    a1 = sess.run(prediction1, feed_dict={x: a, keep_prob: 1.0})
    
    
    pd.DataFrame({'imageid': Imageid, 'label': a1}).set_index('imageid').to_csv('sub4.csv')
    
        #print(sess.run(accuracy, feed_dict = {x: mnist.test.images, y : mnist.test.labels}))


train_neural_network(x)